# 01 - View Preprocessing

This notebook visualizes and aids the design of the preprocessing steps of refining the Diabetes Dataset.

This notebook does **NOT** perform the actual preprocessing; that is handled by `build_dataset.py` 

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import pandas as pd
import sys

import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns
plt.style.use('ggplot')
sns.set()
# plt.style.use('seaborn')
mpl.style.use('seaborn')

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999
pd.options.display.max_rows = 500
np.set_printoptions(precision=6)

DATA_PATH = '../data/'

print(pd.__version__)

0.23.4
